In [3]:
import torch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion
import pandas as pd
from PIL import Image
import os
import io
import base64

# Load the DataFrame
df = pd.read_parquet("hf://datasets/YaYaB/onepiece-blip-captions/data/train-00000-of-00001-7e86e8a67581ad82.parquet")

# Directory to save images
output_folder = "images2"
os.makedirs(output_folder, exist_ok=True)

# Iterate over rows to save each image
for idx, row in df.iterrows():
    # Assuming 'image' column contains image data in base64 format or as bytes
    image_data = row['image'].get('bytes')  # Modify if necessary based on actual format (e.g., if it's already binary)
    
    # Decode if base64 encoded (skip this step if images are already in bytes format)
    if isinstance(image_data, str):  
        image_data = base64.b64decode(image_data)
    
    # Convert bytes data to an Image object
    image = Image.open(io.BytesIO(image_data))
    
    # Save image to output folder with a unique name
    image_filename = os.path.join(output_folder, f"image_{idx}.png")
    image.save(image_filename)

'''
Use One piece to train a Gaussian Diffusion model
Figure out how to get the images onto your computer.
Learn the architecture and minimize the source file to only the essential pieces of code.
*Try using local PC. The original DDPM was only 34M parameters
'''

'\nUse One piece to train a Gaussian Diffusion model\nFigure out how to get the images onto your computer.\nLearn the architecture and minimize the source file to only the essential pieces of code.\n*Try using local PC. The original DDPM was only 34M parameters\n'

In [3]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    flash_attn = True
)

diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 1000    # number of steps
)

training_images = torch.rand(8, 3, 128, 128) # images are normalized from 0 to 1
loss = diffusion(training_images)
loss.backward()

# after a lot of training

sampled_images = diffusion.sample(batch_size = 4)
sampled_images.shape # (4, 3, 128, 128)

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


C:\Users\jesse\miniconda3\envs\test2\Lib\site-packages\torch\backends\cuda\__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(


sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
import pandas as pd
from PIL import Image, ImageOps
import os
import io

# Load the DataFrame
df = pd.read_parquet("hf://datasets/YaYaB/onepiece-blip-captions/data/train-00000-of-00001-7e86e8a67581ad82.parquet")

# Directories to save images
original_output_folder = "images_folder"
resized_output_folder = "images128"
os.makedirs(original_output_folder, exist_ok=True)
os.makedirs(resized_output_folder, exist_ok=True)

# Iterate over rows to process each image
for idx, row in df.iterrows():
    # Access the image bytes
    image_data = row['image'].get('bytes')
    
    # Open the image from bytes
    image = Image.open(io.BytesIO(image_data))
    
    # Resize image, maintaining aspect ratio, then pad to 128x128 if needed
    image.thumbnail((128, 128), Image.LANCZOS)  # Resize with aspect ratio
    
    # Pad the image to make it exactly 128x128
    image_padded = ImageOps.pad(image, (128, 128), color=(255, 255, 255))  # White padding, adjust color as needed
    
    # Save original and resized images
    original_filename = os.path.join(original_output_folder, f"image_{idx}.png")
    resized_filename = os.path.join(resized_output_folder, f"image_{idx}.png")
    image.save(original_filename)
    image_padded.save(resized_filename)


In [2]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    flash_attn = True
)

diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 1000,           # number of steps
    sampling_timesteps = 250    # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
)

trainer = Trainer(
    diffusion,
    'images128',
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 700000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = True              # whether to calculate fid during training
)

trainer.train()
checkpoint_path = "checkpoint.pth"
torch.save(model.state_dict(), checkpoint_path)
print(f"Model checkpoint saved to {checkpoint_path}")

C:\Users\jesse\miniconda3\envs\test2\Lib\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to C:\Users\jesse/.cache\torch\hub\checkpoints\pt_inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:04<00:00, 20.6MB/s]


  0%|          | 0/700000 [00:00<?, ?it/s]

C:\Users\jesse\miniconda3\envs\test2\Lib\site-packages\torch\backends\cuda\__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
C:\Users\jesse\OneDrive\Desktop\Independent Study\Code\denoising-diffusion-pytorch\denoising_diffusion_pytorch\attend.py:88: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  out = F.scaled_dot_product_attention(


OutOfMemoryError: CUDA out of memory. Tried to allocate 130.00 MiB. GPU 

In [1]:
import torch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion

# Define the model architecture (same as in training)
model = Unet(
    dim=64,
    dim_mults=(1, 2, 4, 8),
    flash_attn=True
)

# Define the diffusion process
diffusion = GaussianDiffusion(
    model,
    image_size=128,
    timesteps=1000,
    sampling_timesteps=250  # using DDIM sampling for faster inference
)

# Load the trained model checkpoint (replace with your checkpoint path)
checkpoint_path = "checkpoint.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location="cpu"))
model.eval()  # Set the model to evaluation mode

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint.pth'

In [ ]:
import os
from PIL import Image

# Directory to save generated samples
output_folder = "generated_images"
os.makedirs(output_folder, exist_ok=True)

# Generate a batch of samples
with torch.no_grad():
    sampled_images = diffusion.sample(batch_size=4)  # (4, 3, 128, 128) tensor

# Save generated images
for i, img_tensor in enumerate(sampled_images):
    img_array = (img_tensor.permute(1, 2, 0).numpy() * 255).astype("uint8")  # Convert to image format
    img = Image.fromarray(img_array)
    img.save(os.path.join(output_folder, f"generated_image_{i}.png"))
